# API Telegram Chat Data

Caderno de Desenvolvimento <br>
Curadoria de [Rafael Barbosa](https://www.linkedin.com/in/barbosa89/)

---

## 1\. Telegram

O **Telegram** representa a fonte transacional de dados.

Vamos **criar um *bot*** e **criar um grupo** com *bot* adicionado.

 **Todas** as mensagens enviadas no grupo serão captadas pelo *bot* e trabalhadas através da API (*application programming interface*) de *bots* do **Telegram** (documentação neste [link](https://core.telegram.org/bots/api)).

### **1.1.** Criar um *bot* no Telegram:

- Faça login na versão `web` do aplicativo através desse [link](https://web.telegram.org). Uma conta ativa será necessária para este passo.

- Na barra de **Pesquisa** digite `BotFather`, abra o *chat* marcado como verificado, e aperte "start";

![](https://github.com/rafie-b/Data-Warehouse-AWS-Pipeline-Chat-API/blob/main/repo-API-Telegram-Chat-Data/abrir-chat-botfather.png?raw=true)

- No chat, retorne `/newbot`, o nome do *bot* e o nome de usuário do *bot* (precisa terminar com sufixo `_bot`);

![](https://github.com/rafie-b/Data-Warehouse-AWS-Pipeline-Chat-API/blob/main/repo-API-Telegram-Chat-Data/criar-novo-bot.png?raw=true)

- Salve o `token` de acesso da API HTTP em local seguro.

### **1.2.** Criar um grupo e descrevê-lo, adicionar o *bot* como administrador e desabilitá-lo para novos grupos:

- Aperte o botão com o ícone de um "lápis" e selecione `New Group`;

![](https://github.com/rafie-b/Data-Warehouse-AWS-Pipeline-Chat-API/blob/main/repo-API-Telegram-Chat-Data/criar-grupo.png?raw=true)

- Digite e selecione o nome do *bot* recém criado, aperte no ícone de seta, digite o nome do grupo e aperte na seta novamente.

![](https://github.com/rafie-b/Data-Warehouse-AWS-Pipeline-Chat-API/blob/main/repo-API-Telegram-Chat-Data/novo-grupo-bot.png?raw=true)

- Abra o *chat* do grupo, aperte no ícone do grupo, aperte no ícone com um lápis para editar, no campo de descrição escreva **"Atenção, todas as mensagens são armazenadas pelo *bot* do grupo"**, e aperte o botão de confirmação para salvar as alterações;

![](https://github.com/rafie-b/Data-Warehouse-AWS-Pipeline-Chat-API/blob/main/repo-API-Telegram-Chat-Data/add-descri%C3%A7%C3%A3o.png?raw=true)

- Selecione `Administrators`, aperte no ícone para adiocionar um usuário, selecione o *bot* e aperte o botão de *check*, ou "V", para salvar.

![](https://github.com/rafie-b/Data-Warehouse-AWS-Pipeline-Chat-API/blob/main/repo-API-Telegram-Chat-Data/add-bot-admin.png?raw=true)

- Para que ele não possa ser adicionado a outros grupos abra o *chat* com o **`BotFather`**, digite `/mybots` e selecione o *bot* pelo seu nome de usuário;

![](https://github.com/rafie-b/Data-Warehouse-AWS-Pipeline-Chat-API/blob/main/repo-API-Telegram-Chat-Data/my-bots.png?raw=true)

- Selecione `Bot Settings`, `Allow Groups?` e `Turn groups off`.

![](https://github.com/rafie-b/Data-Warehouse-AWS-Pipeline-Chat-API/blob/main/repo-API-Telegram-Chat-Data/desliga-grupos.png?raw=true)

### **1.3.** Extrair dados das mensagens do grupo através da API

Enviar mensagens no grupo (text, imagem, arquivos, video, áudio, etc.) e as consumir através da API de *bots* do **Telegram**.

> **Nota:** A documentação completa da API pode ser encontrada neste [link](https://core.telegram.org/bots/api)

In [1]:
### guarda o API token do bot

from getpass import getpass

print('Digite o API token do Telegram bot')
token = getpass()

### guarda base_url comum para todos os métodos da API

base_url = f'https://api.telegram.org/bot{token}'

Digite o API token do Telegram bot
··········


In [2]:
### O método getMe retorna informações sobre o bot

import requests
import json

response_me = requests.get(url=f'{base_url}/getMe')
print(json.dumps(json.loads(response_me.text), indent=2))

{
  "ok": true,
  "result": {
    "id": 7084060512,
    "is_bot": true,
    "first_name": "ebac_m42_bot",
    "username": "ebacm42_bot",
    "can_join_groups": false,
    "can_read_all_group_messages": false,
    "supports_inline_queries": false
  }
}


In [3]:
### O método getUpdates retorna as mensagens e suas informações captadas pelo bot

from abc import update_abstractmethods

response = requests.get(url=f'{base_url}/getUpdates')

# print(json.dumps(json.loads(response.text), indent=2))

In [4]:
### carrega no dict "parsed_data" as informacoes extraidas atraves da API
from datetime import datetime
import json

### cria dict "parsed_data" vazio onde as informacoes de mensagens serão alocadas
parsed_data = dict()

## escreve o conteudo de getUpdates no telegram.json
with open("telegram.json", "w") as file:
  file.write(json.dumps(json.loads(response.text), indent=2))

## carrega telegram.json no dict "updates"
with open('telegram.json', mode='r', encoding='utf8') as fp:
  updates = json.load(fp)

### define variáveis de data de extracao de dados
date = datetime.now().strftime('%Y-%m-%d')
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

### varredura em todos os itens do dict "result"
for data in updates['result']:
    data = data["message"]

    parser = dict() # dict para armazenar chaves e valores de cada iteracao

    ### varredura em todos os itens do dict "message"
    for key, value in data.items():
        if key == 'from':
            for k, v in data[key].items():  ### varredura em todos os itens do dict "from"
                if k in ['id', 'is_bot', 'first_name']:
                  parser[f"{key if key == 'chat' else 'user'}_{k}"] = [v]

        elif key == 'chat':
            for k, v in data[key].items(): ### varredura em todos os itens do dict "chat"
                if k in ['id', 'type']:
                  parser[f"{key if key == 'chat' else 'user'}_{k}"] = [v]

        elif key in ['message_id', 'date', 'text']:
            parser[key] = [value]

    ### se a chave "text" nao for encontrada o valor None sera atribuido a chave "text"
    if not 'text' in parser.keys():
      parser['text'] = [None]

    ### carrega as chaves de data de processamento
    parser['context_date'] = [date]
    parser['context_timestamp'] = [timestamp]

    ### atualiza o dict "parsed_data" adicionando os valores da iteracao a cada chave correspondente
    for key, value in parser.items():
      if key in parsed_data:
          parsed_data[key] += value  # Extend the existing list with new values
      else:
          parsed_data[key] = value  # if doesnt exist add the new key-value pair to the existing dictionary

In [5]:
### visualiza os dados armazenados no dict "parsed_data"
print(type(parsed_data))
for k, v in parsed_data.items():
  print(f"{k}: {v}")

<class 'dict'>
message_id: [14, 15, 16, 17, 18, 19, 20]
user_id: [7162819601, 7162819601, 7162819601, 7162819601, 7162819601, 7162819601, 7162819601]
user_is_bot: [False, False, False, False, False, False, False]
user_first_name: ['Rafie', 'Rafie', 'Rafie', 'Rafie', 'Rafie', 'Rafie', 'Rafie']
chat_id: [-1001993072220, -1001993072220, -1001993072220, -1001993072220, -1001993072220, -1001993072220, -1001993072220]
chat_type: ['supergroup', 'supergroup', 'supergroup', 'supergroup', 'supergroup', 'supergroup', 'supergroup']
date: [1710613562, 1710613607, 1710613696, 1710613707, 1710613714, 1710614580, 1710620050]
text: [None, None, None, 'messages', 'from newday', None, 'exercise finish']
context_date: ['2024-03-16', '2024-03-16', '2024-03-16', '2024-03-16', '2024-03-16', '2024-03-16', '2024-03-16']
context_timestamp: ['2024-03-16 23:45:21', '2024-03-16 23:45:21', '2024-03-16 23:45:21', '2024-03-16 23:45:21', '2024-03-16 23:45:21', '2024-03-16 23:45:21', '2024-03-16 23:45:21']


In [6]:
### cria uma tabela pyArrow a partir do dict "parsed_data"
import pyarrow as pa

table = pa.Table.from_pydict(mapping=parsed_data)

print(type(table))
table

<class 'pyarrow.lib.Table'>


pyarrow.Table
message_id: int64
user_id: int64
user_is_bot: bool
user_first_name: string
chat_id: int64
chat_type: string
date: int64
text: string
context_date: string
context_timestamp: string
----
message_id: [[14,15,16,17,18,19,20]]
user_id: [[7162819601,7162819601,7162819601,7162819601,7162819601,7162819601,7162819601]]
user_is_bot: [[false,false,false,false,false,false,false]]
user_first_name: [["Rafie","Rafie","Rafie","Rafie","Rafie","Rafie","Rafie"]]
chat_id: [[-1001993072220,-1001993072220,-1001993072220,-1001993072220,-1001993072220,-1001993072220,-1001993072220]]
chat_type: [["supergroup","supergroup","supergroup","supergroup","supergroup","supergroup","supergroup"]]
date: [[1710613562,1710613607,1710613696,1710613707,1710613714,1710614580,1710620050]]
text: [[null,null,null,"messages","from newday",null,"exercise finish"]]
context_date: [["2024-03-16","2024-03-16","2024-03-16","2024-03-16","2024-03-16","2024-03-16","2024-03-16"]]
context_timestamp: [["2024-03-16 23:45:21","2